# Colab的默认环境跟这里的默认环境有差异，需要安装额外的依赖

In [1]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 69 not upgraded.
Need to get 2129 kB of archives.
After this operation, 7662 kB of additional disk space will be used.
Get:1 http://mirrors.tuna.tsinghua.edu.cn/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2129 kB]
Fetched 2129 kB in 2s (1374 kB/s)  
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
Selecting previously unselected package git-lfs.
(Reading database ... 52181 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...


In [2]:
!pip install wget
!pip install transformers
!pip install bitsandbytes
!pip install jax
!apt-get update
!apt-get install -y p7zip-full
!apt-get install -y iputils-ping


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=40f9338a5a5d28920cb17ec1fd2810fd9d0ebd59cd4b2232ff1f754a3f171a30
  Stored in directory: /root/.cache/pip/wheels/df/d8/7c/a62984d928c4d9d2d2abe86ae3bd091679cfc87bf6709256df
Successfully built wget
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 121.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 125.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 141.0 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 3.3 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━

# 创建Temp文件夹

In [3]:
import wget
import os,sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxyCLI,proxyWget
cb=setProxyCLI()
proxy=cb['proxy']
region=cb['region']
proxyURL=cb['proxyURL']

# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
def create_temp_folder(temp_folder):
  if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# 获取当前运行的 ipynb 文件所在的目录
cwd = os.getcwd()

# 在该目录下创建临时文件夹 temp
temp_folder = os.path.join(cwd, 'temp')

# 如果temp文件夹不存在，则创建文件夹
create_temp_folder(temp_folder)

# 覆盖下载 fast-DreamBooth.ipynb 文件到 temp 文件夹中
file_name='fast-DreamBooth.ipynb'
fastDreamBoothPath = os.path.join(temp_folder, file_name)
if os.path.exists(fastDreamBoothPath):
    os.remove(fastDreamBoothPath)
    
fastDreamBoothPath = proxyWget(
    url="https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/fast-DreamBooth.ipynb", 
    out=os.path.join(temp_folder, file_name), 
    proxyURL=proxyURL)

print(fastDreamBoothPath)

Ping程序运行耗时: 15毫秒
Ping通该IP与端口：alchemist-experience:7890，正在使用【贝式】代理
/output/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth.ipynb


# 清理Notebook中的输出，对Colab版中的路径进行替换，以适应openbayes或autodl的路径

In [19]:
import json
import os
import re

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-modified" + file_ext
  return modified_file_path

def replace_root_path(notebook: str, old_root: str, new_root: str) -> str:
    # 加载notebook
    with open(notebook, "r") as f:
        nb = json.load(f)

    # 替换旧的根路径为新的根路径
  
    for cell in nb["cells"]:            
        if cell["cell_type"] == "code":
            # Initialize the flag variable
            import_added = False  
            # 隐藏代码
            cell["metadata"] = {"collapsed": True}
            # 清除代码单元格的执行次数
            cell['execution_count'] = None
            # 清除代码单元格的输出
            cell['outputs'] = []            
            for i, line in enumerate(cell["source"]):
                # 替换@param{type: 'xxx'}中的单引号为双引号
                if "@param" in line:
                    cell["source"][i] = line.replace("'", '"')           
                # 添加一个条件判断，只有当该行不是网址路径时才进行替换
                if not line.startswith(("http", "https")):
                    cell["source"][i] = line.replace(old_root, new_root)
                # 把引入google.colab的库的代码行标注为注释
                if "google.colab" in line:
                    cell["source"][i] = "# " + line
                # 把使用google drive盘挂载的代码行标注为注释
                if "drive.mount" in line:
                    cell["source"][i] = "# " + line
                 # 如果找到 "!wget" 字符串，就将其替换为 "!proxy && wget"
                if '!wget' in line:
                    cell["source"][i] = line.replace('!wget', '!proxy && wget')
                # 定义 proxyWget 函数（请确保已经定义了 proxyWget 函数）

                import textwrap

                if "wget.download" in line:
                    # 使用正则表达式查找所有使用了 proxyWget 函数的行
                    match = re.search(r"wget\.download\(.+\)", line)
                    if match:
                        # 获取 wget.download 函数调用的参数
                        params = match.group().strip("wget.download()")
                        # 计算新的一行代码的缩进
                        indent = " " * line.count(" ")
                        # 将 wget.download 函数调用替换为带有新参数的函数调用
                        new_line = f"{indent}proxyWget({params}, os.getcwd(), '{proxyURL}')\n"
                        cell["source"][i] = new_line

                        
            for i, line in enumerate(cell["source"]):
                # Check if the line contains "wget" and add the import statement if necessary
                if "wget" in line and not import_added:
                    import_statements = [
                        "import sys",
                        "from IPython.utils import capture",
                        "sys.path.append('../../')", 
                        "from func.env import setProxyCLI,proxyWget", 
                        "cb=setProxyCLI()", 
                        "proxy=cb['proxy']",
                        "region=cb['region']", 
                        "proxyURL=cb['proxyURL']"
                        ,""
                    ]
                    import_statements_with_newline = [s + "\n" for s in import_statements]
                    # print(import_statements_with_newline)
                    cell["source"] = import_statements_with_newline + cell["source"]
                    # Set the flag to indicate that the import has been added
                    import_added = True
                
                # # 原始字符串
                # original = "!cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/"
                # if original in line:
                #     # 替换python路径
                #     cell["source"][i] = re.sub(r'/usr/local/lib/python3.8/', '/usr/local/lib/python/', original)
                    
                    
    # 保存修改后的notebook
    newNotebook=modify_file_name(notebook)
    with open(newNotebook, "w") as f:
        json.dump(nb, f)
        
    return newNotebook

def check_environment(output):
    # 检查输出是否包含字符串"autodl"
    # print(output)
    global content_path
    for i in output:
        if "AutoDL" in i:
            print("当前运行环境为autodl")
            content_path = "/root/content"
            break
        # 检查输出是否包含字符串"openbayes"
        elif "OpenBayes" in i:
            print("当前运行环境为openbayes")
            content_path = "/openbayes/home/content"
            break
    return content_path

# 尝试执行第一个命令并存储输出
try:
    output1 = !cd /openbayes/home && /etc/welcome
    content_path = check_environment(output1)
except Exception as e:
    print("无法执行命令：", e)
    
# 尝试执行第二个命令并存储输出
try:
    output2 = !source /etc/autodl-motd
    content_path = check_environment(output2)
except Exception as e:
    print("无法执行命令：", e)
    
print("内容路径：", content_path)

# 在根目录下创建Content文件夹
content_folder = content_path
# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

newNotebookPath=replace_root_path(
    notebook=fastDreamBoothPath,
    old_root="/content",
    new_root=content_folder
)
print(newNotebookPath)

当前运行环境为openbayes
无法执行命令： local variable 'content_path' referenced before assignment
内容路径： /openbayes/home/content
/output/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth-modified.ipynb


In [8]:
!cd /openbayes/home && /etc/welcome

OpenBayes

目录说明

- /output 工作空间内的数据保存地址，容器停止后，该目录中的内容不会被删除
- /openbayes/home 为 /output 的软链路径，在程序中与上方的路径效果相同
- /input0 - /input4 为数据集绑定地址，此地址为只读权限，不会占用工作空间的存储容量，最多支持同时绑定 5 个

⚠️  其他目录下的内容在容器关闭后会被自动删除！更多信息请访问 https://openbayes.com/docs/concepts 

使用命令行工具

通过命令 bayes gear init 可以初始化容器中的命令行工具，更多信息见 https://openbayes.com/docs/cli/



# 对指定的变量名进行字符串和参数的替换

In [5]:
import json

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-customData" + file_ext
  return modified_file_path

# 读取修改后的notebook
def update_ipynb_vars(ipynb_file, var_map):
  with open(ipynb_file, 'r') as f:
    notebook = json.load(f)
  
  # 用更新后的值替换变量赋值
  for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
      for i in range(len(cell['source'])):
        for var, val in var_map.items():
          if isinstance(val, str):
            val = f'"{val}"'
          # 有 #@ 说明这一行是提供给用户自行填充的参数，所以才进行参数替换
          identifier="#@"
          thisLine=cell['source'][i]
          if identifier in thisLine and (thisLine.startswith(f'{var} =') or thisLine.startswith(f'{var}=')):
            annotation=thisLine.split(identifier)[1]
            cell['source'][i] = f'{var}={val} {identifier+annotation}' 

  # 保存修改后的notebook
  updatedDataNotebookPath=modify_file_name(ipynb_file)
  with open(updatedDataNotebookPath, 'w') as f:
    json.dump(notebook, f)

  return updatedDataNotebookPath

# 更新变量
replacements = {
  'Model_Version': 'V2.1-768px',
  'Session_Name': 'FuXingHao768',
  'Crop_size': 768,
  'Remove_existing_instance_images': False,
  'IMAGES_FOLDER_OPTIONAL':"/openbayes/home/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/FuXingHao768/instance_images",
  'Crop_images':False,
  'Resolution': 768,
}

# 更新变量
updatedDataNotebookPath=update_ipynb_vars(newNotebookPath, replacements)

# 检查最终版本的ipynb里面的赋值情况

In [6]:
import json
import os

def print_with_color(string, keywords):
    # 创建一个空字符串用于存储结果
    result = ""

    # 遍历字符串中的每一个字眼
    for word in string.split(" "):
        if os.path.isabs(word.strip('"')):
            result += f"\033[0;42m{word}"
        elif word.isnumeric():
            result += f"\033[0;36m{word}"
        else:
            # 如果这个字眼是关键词，就将它的颜色更改为相应的颜色
            if word in keywords:
                result += f"{keywords[word]}{word}"
            # 否则，使用默认颜色
            else:
                result += f"\033[1;33m{word}"
        result += ' \033[0m' # 收尾都要加一个' \033[0m'
    return result


keywords = {
    "=": "\033[1;34m",# &&：正常颜色
    "False": "\033[1;31m",# git：红色
    "True": "\033[0;32m",# user：绿色
    "config": "\033[1;33m",# config：黄色
    "--global": "\033[1;34m",# --global：蓝色
    '""': "\033[1;35m",# http.proxy：紫色
    "user.email": "\033[1;36m",# user.email：深蓝色
    "user.name": "\033[1;37m",# user.name：深绿色
    "http.proxy": "\033[1;38m",# http.proxy：深紫色
    "proxyURL": "\033[1;39m",# proxyURL：深红色
    "cd": "\033[1;31m",# cd：红色
    "dir_path": "\033[1;41m",# dir_path：深黑色
}

# colored_string = print_with_color("git config --global user.email user@example.com", keywords)
# print(colored_string)


def detect_constant_assignments(ipynb_file: str):
    # 定义一个 set，用于记录已经出现过的字符串
    seen_strings = set()
    empty_strings = set()
    # 打开文件并读取数据
    with open(ipynb_file, 'r') as f:
        ipynb_data = json.load(f)

    # 遍历所有的代码单元
    for cell in ipynb_data['cells']:
        if cell['cell_type'] == 'code':
            # 遍历当前代码单元中的所有行
            for line in cell['source']:
                # 检测是否是赋值行，并获取变量名和值
                parts = line.split('=')
                if len(parts) == 2 and "@param" in line:
                    var_name = parts[0].strip()
                    value = parts[1].strip()
                    before, after = value.split('#@param')
                    # 如果 parts[0] 还没有出现过，就输出
                    if parts[0] not in seen_strings:
                        seen_strings.add(parts[0])
                        print(
                            f'\033[1;37m{var_name}' + 
                            f'{ print_with_color(f" = {before}",keywords) }' + 
                            '\033[1;30m' + 
                            f'#@param{after}'
                        )
                    
                    
detect_constant_assignments(updatedDataNotebookPath)


# print("\033[1;33m \t\t\tprint颜色参考 \033[0m \n");

# print("\033[1;33m \t 字体颜色:\n");
# print("\033[0m none:\n");
# print("\033[0;30m back:\n");
# print("\033[1;30m dark_back:\n");
# print("\033[0;34m blue:\n");
# print("\033[1;34m light_blue:\n");
# print("\033[0;32m green:\n");
# print("\033[1;32m light_green:\n");
# print("\033[0;36m cyan:\n");
# print("\033[1;36m light_scan:\n");
# print("\033[0;31m red:\n");
# print("\033[1;31m light_read:\n");
# print("\033[0;35m purple:\n");
# print("\033[1;35m light_purple:\n");
# print("\033[0;33m brown:\n");
# print("\033[1;33m yellow:\n");
# print("\033[0;37m light_yellow:\n");
# print("\033[1;37m white:\n");

# print("\033[1;33m \t 背景颜色:\n");
# print("\033[0m none:\n");
# print("\033[0;40m back:\n");
# print("\033[0;44m blue:\n");
# print("\033[0;42m green:\n");
# print("\033[0;46m cyan:\n");
# print("\033[0;41m red:\n");
# print("\033[0;45m purple:\n");
# print("\033[0;43m brown:\n");
# print("\033[0;47m light_yellow:\033[0m\n");

# print("\033[1;33m \t 背景字体颜色:\n");
# print("\033[47;31mhello world\033[?25l");    



Model_Version = "V2.1-768px"  #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Huggingface_Token = ""  #@param {type:"string"}
Custom_Model_Version = "1.5"  #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Path_to_HuggingFace = ""  #@param {type:"string"}
CKPT_Path = ""  #@param {type:"string"}
CKPT_Link = ""  #@param {type:"string"}
Session_Name = "FuXingHao768"  #@param{type: 'string'}
Session_Link_optional = ""  #@param{type: 'string'}
Remove_existing_instance_images = False  #@param{type: 'boolean'}
IMAGES_FOLDER_OPTIONAL = "/openbayes/home/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/FuXingHao768/instance_images"  #@param{type: 'string'}
Crop_images = False  #@param{type: 'boolean'}
Crop_size = 768  #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"]
Remove_existing_concept_images = True  #@param{type: 'boolean'}
Resume_Training = False  #@param {type:"boolean"}
UNet_Training_Steps = 3000  #@param{type: 'number'}
Text_Encoder_Training_Steps = 350  #@param{type: 'nu